In [39]:
import glob 
import pandas as pd
import os

filenames = glob.glob('../backend/data/cleaned/*.csv')

arr = [] 

for idx, fname in enumerate(filenames) :
    full_path = os.path.abspath(fname)
    df = pd.read_csv(full_path, on_bad_lines='skip')
    arr.append(df)
    print(f"File {idx} : {fname} has been read successfully")



print(arr[2])


# for "Year" and "Code" Columns
# if float convert to int
# if string convert to int
for df in arr :
    
    for col in df.columns :
        if col == "Year":
            if df[col].dtype == "float64" :
                df[col] = df[col].astype(int)
            elif df[col].dtype == "object" :
                df[col] = df[col].astype(int)
                
    df.columns = df.columns.str.strip()
    
    for col in df.select_dtypes(include=["object"]).columns:
        df[col] = df[col].str.strip()
    
                
                
    



merged_df = arr[0]

index = 0 
for df in arr[1:] :
    print("DTYPES BEFORE MERGING : ")
    print(merged_df.dtypes, 'df.dtypes')
    print("------------------------------------------")
    print("MERGING: ", filenames[index])
    print("#########################################################################################################")
    print("MERGING DTYPES")
    print(df.dtypes, 'df.dtypes')
    print("#########################################################################################################")
    
    merged_df = pd.merge(merged_df, df, on=['Year', 'Code'], how = 'outer', suffixes = (f'_left{index}', f'_right{index}'))
    
    # print(merged_df)
    index+= 1

merged_df = merged_df.drop_duplicates()

print("FINAL DF")
merged_df.columns




File 0 : ../backend/data/cleaned\cleaned_GDP.csv has been read successfully
File 1 : ../backend/data/cleaned\cleaned_pct_renewable_energy.csv has been read successfully
File 2 : ../backend/data/cleaned\cleaned_renewable_output_gwh.csv has been read successfully
File 3 : ../backend/data/cleaned\country_co2_emissions.csv has been read successfully
File 4 : ../backend/data/cleaned\country_co2_emissions_fossil_land.csv has been read successfully
File 5 : ../backend/data/cleaned\emissions_merged_no2_ch4_.csv has been read successfully
          Country Code    Year Renewable_Output_GWh
0     Afghanistan  AFG  1990.0                  764
1     Afghanistan  AFG  1991.0                  690
2     Afghanistan  AFG  1992.0                  478
3     Afghanistan  AFG  1993.0                  475
4     Afghanistan  AFG  1994.0                  472
...           ...  ...     ...                  ...
6729        World  GWL  2011.0                   ..
6730        World  GWL  2012.0                  

Index(['Entity_left0', 'Code', 'Year', 'GDP growth (annual %)',
       'Entity_right0', 'Renewable_Share', 'Country', 'Renewable_Output_GWh',
       'Entity_left3', 'Annual CO2 emissions', 'Entity_right3',
       'Annual CO2 emissions from land-use change',
       'Annual CO2 emissions from fossil fuels and industry', 'gdp',
       'nitrous_oxide_per_capita', 'methane_per_capita', 'gas_co2_per_capita',
       'total_ghg', 'country', 'co2_per_capita', 'population', 'gdp.1',
       'temperature_change_from_ch4', 'temperature_change_from_ghg',
       'temperature_change_from_co2', 'temperature_change_from_n2o',
       'land_use_change_co2_per_capita', 'co2', 'methane', 'nitrous_oxide',
       'total_ghg.1', 'Country Name', 'Country Code', 'Indicator Name',
       'Indicator Code', 'emissions'],
      dtype='object')

In [40]:
# CELL TO REMOVE UNNEEDED COLUMNS & MODIFY COLUMN NAMES

merged_df.drop(columns=['Indicator Name', 'Indicator Code', 'Country Name', 'Country Code'], inplace= True)

merged_df.rename(columns = {'emissions' : 'HFC/PFC/SF6'}, inplace = True)

# drop all rows where year < 1990

merged_df.drop(merged_df[merged_df['Year'] < 1990].index, inplace = True)

# replace null values with 0

# merged_df.fillna(0, inplace = True)



In [41]:
merged_df.to_csv('output/merged_all.csv', index=False)